In [1]:
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [4]:
data <- read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSTyupJYW8-dOI7rTKOVE-bMT4M_8UnVnRLOJGavyVh33-4wHTkP4Q-uWX9mwMQul2kxMl8aJLR0FHB/pub?output=csv")

Parsed with column specification:
cols(
  .default = col_double(),
  url = col_character()
)

See spec(...) for full column specifications.



In [5]:
#renaming data
renamed_data <- data %>%
    rename(monday = weekday_is_monday,
        tuesday = weekday_is_tuesday,
        wednesday = weekday_is_wednesday,
        thursday = weekday_is_thursday,
        friday = weekday_is_friday,
        saturday = weekday_is_saturday,
        sunday = weekday_is_sunday,
        lifestyle = data_channel_is_lifestyle,
        entertainment =  data_channel_is_entertainment,
        business = data_channel_is_bus,
        society_and_medicine =  data_channel_is_socmed,
        technology =  data_channel_is_tech,
        world =  data_channel_is_world)
head(renamed_data)

url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,⋯,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
http://mashable.com/2013/01/07/amazon-instant-video-browser/,731,12,219,0.6635945,1,0.8153846,4,2,1,⋯,0.10000000,0.7,-0.3500000,-0.600,-0.2000000,0.5000000,-0.1875000,0.00000000,0.1875000,593
http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/,731,9,255,0.6047431,1,0.7919463,3,1,1,⋯,0.03333333,0.7,-0.1187500,-0.125,-0.1000000,0.0000000,0.0000000,0.50000000,0.0000000,711
http://mashable.com/2013/01/07/apple-40-billion-app-downloads/,731,9,211,0.5751295,1,0.6638655,3,1,1,⋯,0.10000000,1.0,-0.4666667,-0.800,-0.1333333,0.0000000,0.0000000,0.50000000,0.0000000,1500
http://mashable.com/2013/01/07/astronaut-notre-dame-bcs/,731,9,531,0.5037879,1,0.6656347,9,0,1,⋯,0.13636364,0.8,-0.3696970,-0.600,-0.1666667,0.0000000,0.0000000,0.50000000,0.0000000,1200
http://mashable.com/2013/01/07/att-u-verse-apps/,731,13,1072,0.4156456,1,0.5408895,19,19,20,⋯,0.03333333,1.0,-0.2201923,-0.500,-0.0500000,0.4545455,0.1363636,0.04545455,0.1363636,505
http://mashable.com/2013/01/07/beewi-smart-toys/,731,10,370,0.5598886,1,0.6981982,2,2,0,⋯,0.13636364,0.6,-0.1950000,-0.400,-0.1000000,0.6428571,0.2142857,0.14285714,0.2142857,855


In [6]:
#tidying data
tidied_data <- renamed_data %>%
    pivot_longer(cols =  lifestyle:world, 
                 names_to = 'Genre', 
                 values_to = 'Genre_Present') %>%
    filter(Genre_Present == 1) %>%
    pivot_longer(cols =   monday:sunday, 
                 names_to = 'Day_Of_Week', 
                 values_to = 'Day_Present') %>%
    filter(Day_Present == 1) %>%
    mutate(abs_pos_neg_ratio = (global_rate_positive_words*avg_positive_polarity)/abs(global_rate_negative_words*avg_negative_polarity)) %>%
    filter(global_rate_positive_words != 0 & global_rate_negative_words != 0) %>%
    mutate(Genre = as_factor(Genre)) %>%
    mutate(Day_Of_Week = as_factor(Day_Of_Week)) %>%
    mutate(n_tokens_content = as.numeric(n_tokens_content))%>%
    select(Genre, Day_Of_Week, shares, n_tokens_content)

head(tidied_data)

Genre,Day_Of_Week,shares,n_tokens_content
<fct>,<fct>,<dbl>,<dbl>
entertainment,monday,593,219
business,monday,711,255
business,monday,1500,211
entertainment,monday,1200,531
technology,monday,505,1072
technology,monday,855,370


In [7]:
#data splitting
set.seed(1234)
data_split <- initial_split(tidied_data, prop = 0.6, strata = shares)
data_training <- training(data_split)
data_testing <- testing(data_split)
data_split

<Analysis/Assess/Total>
<19108/12737/31845>

In [8]:
#grouping into day of week 
day_table <- data_training %>%
    group_by(Day_Of_Week) %>%
    summarize(total_articles=n(), avg_shares=mean(shares),n_tokens_content = mean(n_tokens_content))
day_table

#grouping into article genre
genre_table <- data_training %>%
    group_by(Genre) %>%
    summarize(total_articles=n(), avg_shares=mean(shares),n_tokens_content = mean(n_tokens_content) )
genre_table

`summarise()` ungrouping output (override with `.groups` argument)



Day_Of_Week,total_articles,avg_shares,n_tokens_content
<fct>,<int>,<dbl>,<dbl>
monday,3295,3369.657,610.2589
tuesday,3507,2792.080,610.3738
wednesday,3664,2879.380,588.4479
thursday,3517,2894.013,596.4157
friday,2732,2793.846,590.6116
saturday,1142,3364.944,686.8958
sunday,1251,3531.652,687.8010


`summarise()` ungrouping output (override with `.groups` argument)



Genre,total_articles,avg_shares,n_tokens_content
<fct>,<int>,<dbl>,<dbl>
entertainment,4015,2974.835,642.3584
business,3541,3265.274,559.6238
technology,4146,3178.463,596.2600
lifestyle,1237,3771.733,633.5756
world,4799,2318.699,622.2990
society_and_medicine,1370,3678.478,628.1307


In [9]:
#building model
n_tokens_shares_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>% 
       set_engine("kknn") %>%
       set_mode("regression") 

#building recipe
n_tokens_shares_recipe <- recipe(shares ~ ., data = data_training) %>%
       step_scale(-shares, -Day_Of_Week, -Genre) %>%
       step_center(-shares, -Day_Of_Week, -Genre)



In [11]:
shares_vfold <- vfold_cv(data_training, v = 5, strata = shares)

shares_workflow <- workflow() %>%
       add_recipe(n_tokens_shares_recipe) %>%
       add_model(n_tokens_shares_spec)

gridvals <- tibble(neighbors = 1:100)

shares_results <- shares_workflow %>%
    tune_grid(resamples = shares_vfold, grid = gridvals) %>%
    collect_metrics()

shares_results

neighbors,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,rmse,standard,1.392277e+04,5,1.305276e+03,Model001
1,rsq,standard,4.071848e-04,5,2.043351e-04,Model001
2,rmse,standard,1.317653e+04,5,1.689180e+03,Model002
2,rsq,standard,1.568908e-04,5,7.147482e-05,Model002
3,rmse,standard,1.261885e+04,5,1.629193e+03,Model003
3,rsq,standard,2.192779e-04,5,1.692781e-04,Model003
4,rmse,standard,1.231288e+04,5,1.603565e+03,Model004
4,rsq,standard,3.556967e-04,5,2.908663e-04,Model004
5,rmse,standard,1.191692e+04,5,1.679571e+03,Model005


In [2]:
shares_min <- shares_results %>%
    filter(.metric == "rmse") %>%
    arrange(mean) %>% 
    slice(1)

shares_min

ERROR: Error in shares_results %>% filter(.metric == "rmse") %>% arrange(mean) %>% : could not find function "%>%"


In [3]:
k_min <- shares_min %>%
          pull(neighbors)

shares_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min) %>%
          set_engine("kknn") %>%
          set_mode("regression")

shares_best_fit <- workflow() %>%
          add_recipe(n_tokens_shares_recipe) %>%
          add_model(n_tokens_shares_spec) %>%
          fit(data = data_training)

shares_summary <- shares_best_fit %>%
           predict(data_testing) %>%
           bind_cols(data_testing) %>%
           metrics(truth = shares, estimate = .pred)
shares_summary

ERROR: Error in shares_min %>% pull(neighbors): could not find function "%>%"


In [4]:
shares_preds <- share_best_fit %>%
           predict(data_testing) %>%
           bind_cols(data_testing)

options(repr.plot.width = 8, repr.plot.height = 10)

shares_plot <- ggplot(shares_preds, aes(x = n_tokens_content, y = shares)) +
  geom_point(alpha = 0.4) +
  labs(x="Number of words in the content")+
  geom_line(data = share_preds, aes(x = n_tokens_content, y = .pred), color = "blue")+
  facet_grid(Genre ~ Day_Of_Week)+
  ggtitle("Shares by Number of words in the content")

ERROR: Error in share_best_fit %>% predict(data_testing) %>% bind_cols(data_testing): could not find function "%>%"
